# Hypothesis to test: 
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip
import math
import random
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, \
GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, \
classification_report, make_scorer
import statsmodels.api as sm

# From this project
from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLPv0 import WordBag, AboutMovie


# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

In [2]:
%reload_ext autoreload
word_bag = WordBag()

## Configuration

In [3]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../datasets/'

In [4]:
0.4*360

144.0

## Get users' positive and negative reviews

In [5]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

## Subsample

In [6]:
SAMPLE_FRACTION = 0.4

train_test_dic = {'train': {}, 'test':{}}

for i in ['train','test']:
    for j in ['positive','negative']:
         train_test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .drop(['reviewerName', 'helpful', 'summary', 'unixReviewTime', 'reviewTime'], axis=1)
            

## Create bag of words
Remove accents  
Tokenize  
Lower the case  
Apply custom stop words (keep all negations)  
Remove all non alphabetic characters  
Lematize  
 
Output:  
One list of words for each review 

In [ ]:
# train_test_dic['train']['positive'].iloc[:3,:]

In [ ]:
# for i in ['train','test']:
#     for j in ['positive','negative']:
#         train_test_dic[i][j]['words'] = \
#             word_bag.create(train_test_dic[i][j]['reviewText'])

## Remove reviews that may not be on the movie, but on Amazon/support instead
Input: 
* word tokens 
* one line per review 

In [ ]:
# %reload_ext autoreload
# about_movie = AboutMovie()
# movie_reviews = {'train':{}, 'test':{}}
# for i in ['train','test']:
#     for j in ['positive','negative']:
#          movie_reviews[i][j] = train_test_dic[i][j][[about_movie.check(words) \
#                                                     for words in train_test_dic[i][j]['words']]]

In [ ]:
# train_test_dic['test']['positive'][[not i for i in \
#                                     [about_movie.check(words) for words in train_test_dic[i][j]['words']]]]

In [ ]:
# tot_reviews = 0
# for i in ['train','test']:
#     for j in ['positive','negative']:
#         removed = train_test_dic[i][j].shape[0] - movie_reviews[i][j].shape[0]
#         tot_reviews += train_test_dic[i][j].shape[0]
#         print('Removed {0} ({1:.0%}) {2} {3} reviews'.format(removed, removed / train_test_dic[i][j].shape[0],
#                                                 i, j))

In [ ]:
# import bz2

# if True:
#     with bz2.open(data_path 
#                     + 'movie_reviews_' 
#                     + str(tot_reviews) + 'Pos_Neg_Samples.pkl.bz2'
#                     , "wt") as f:
#          # Write compressed data to file
#         unused = f.write(str(movie_reviews))
    
# if True:
# #     tot_reviews = 4000
#     with bz2.open(data_path 
#                     + 'movie_reviews_' 
#                     + str(tot_reviews) + 'Pos_Neg_Samples.pkl.bz2'
#                     , "rt") as f:
#          # Write compressed data to file
#         test = f.read()
 

In [7]:
df1 = train_test_dic['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

,reviewerID,asin,reviewText,overall
1515619,A32244V7CQUBD6,B00005QFEK,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0


In [8]:
# df1 = movie_reviews['train']['positive']
# df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

NameError: name 'movie_reviews' is not defined

In [ ]:
# movie_reviews = train_test_dic

# For case B

In [10]:
if False:
    pickle_out = open(data_path
                    + 'movie_reviews_' 
                    + str(tot_reviews) + 'Pos_Neg_Samples.pkl'
                    , "wb")
    pickle.dump(movie_reviews, pickle_out)
    pickle_out.close()
 
if True:
    pickle_in = open(data_path
                    + 'reviews_wout_most_subj_0.15.pkl'
                    , "rb")
    movie_reviews = pickle.load(pickle_in)
    pickle_in.close()

In [ ]:
if False:
    for i in ['train','test']:
        for j in ['positive','negative']:
            movie_reviews[i][j]['words'] = \
                word_bag.create(movie_reviews[i][j]['reviewText'])

In [11]:
total = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        print(train_test_dic[i][j].shape)
        total += train_test_dic[i][j].shape[0]
total

(57600, 4)
(57600, 4)
(14400, 4)
(14400, 4)


144000

In [12]:
total = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        print(movie_reviews[i][j].shape)
        total += movie_reviews[i][j].shape[0]
total

(76097, 4)
(74227, 4)
(19075, 4)
(17354, 4)


186753

In [14]:
train_test_dic['train']['positive'].iloc[:,:]

,reviewerID,asin,reviewText,overall
1515619,A32244V7CQUBD6,B00005QFEK,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0
1515627,A32244V7CQUBD6,B00005QFEN,"This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;. There is the continuing plot from an earlier episode of some group on Earth trying to track them down. That's the main reason for Emmanuelle and Hafron to jump to different parts of the world. Otherwise, this episode is mostly sex scenes.",4.0
1515636,A33KKMGGVLZ29T,B00005QFER,"This is an intimate concert of Robert Mirabal. Although I thought that it was, as I said, masterful, the sound, at times sounded a little muffled.The storytelling of the songs gave an insight of native culture and of Mirabal's own family stories and history.The Dance and Ee You Oo are my picks for the best songs, but they are all a joy to watch. The Rare Tribal Mob and the Mirabal Singers/Dancers are great and provide a mesmerising stage performance.Very enjoyable",4.0
1515678,A33P47VEH0YULL,B00005QG2N,"A well put together DVD for the ""Stinkiest band"" in the world. Easy to navigate, and with some pretty interesting side notes...a great collectors item for any fan of Cradle of Filth or Swedish metal.",4.0
1515690,ADG33WELAQRZJ,B00005QG2N,"Great show from the inventors of the extreme gothic metal genre !! 75'of pure mayhem with good visuals and a great sound, although the vocals are a little high in the mix for my taste and the guitar on the right channel is 10 times louder than the one on the left so I ended up switching it to mono to even things up. It's hard to give this more stars esp. when you've watched the PTSFirepower DVD first like I did, that one is so much better. The 5 bonus videoclips totally rule and round up the package making the purchase a worthy one.Now the mockumentary may not be suited to everyone's tastes, personally it bored me and know of a lot of people who feels it ruins the band's image. Well don't watch it then.",4.0
1515691,A3YXITJWFW4BW,B00005QG2N,"I rented this dvd, i didnt buy it. I also rented PanDamonAeon and Mannequin. This dvd has excellent very long concert footage. Its practically the same songs as on Livebait for the Dead. And then we have the rest of the dvd. Im sorry, i love CoF but that ""schockumentary"" thing was just sad. I could barely stand to watch it. It's stupid and boring, maybe if theyd shown more of the band in a more coherent manner...and it just goes on and on forever. This is a 2 hour plus dvd adn half of it is crap. The ""blair twit Project"" is slightly amusing, but mostly boring. The music videos are good though. I would still recommend buying this dvd just for the music videos and the concert footage. PanDamonAeon gave me more joy though.",4.0
1515700,A214NHULS3H7OX,B00005QG2N,"Excellent Sound, Excellent Picture, Tons and Tons of songs. Love the Scorched Earth Erotic video. The only reason I didn't give this 5 stars is because I don't think they let their female singer sing enough, it adds a nice goth touch. I also did not give it five stars because it would have been nice to have the ""her ghost in the fog"" video on here, and finally I thin Dani could have done better on some parts of the songs. For example: On ""her ghost in the fog"" (live version), he chooses to growl in that typical death metal voice, instead of talking in the deep gothic voice like he does on the album. By using this growly voice, I think he is cheese'n off some of the songs best parts. Anyway, these are minor gripes and this is a must have DVD, so definitely BUY IT!...",4.0
1515701,A2IZOU2G1QX0JD,B00005QG2N,"As most of my friends hate, I'm massively into Cradle of Filth. In the style of Napoleon Dynamite describing a Liger, 

## To do: investigate improvements to negative sentences 
For example:  
merge negations with next word, remove next word  
Or lemmatize based on grammar:  https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

### TF-IDF setup

In [ ]:
MAX_FEATURES = 10000

In [ ]:
tfidf = TfidfVectorizer(
    lowercase=True, 
    stop_words=None, 
    max_features=MAX_FEATURES,
    norm='l2',            # normalize each review
    use_idf=True) 

In [ ]:
# def dummy_fun(doc):
#     return doc

# tfidf = TfidfVectorizer(
#     analyzer='word',       # Feed a list of words to TF-IDF
#     tokenizer=dummy_fun,
#     preprocessor=dummy_fun,
#     token_pattern=None,
#     lowercase=False, 
#     stop_words=None, 
#     max_features=MAX_FEATURES,
#     norm='l2',            # normalize each review
#     use_idf=True)        # Keep high weight for most common words

In [ ]:
train_words = pd.concat([movie_reviews['train']['positive']['reviewText'],
                     movie_reviews['train']['negative']['reviewText']])
y_train = np.concatenate([np.ones((movie_reviews['train']['positive'].shape[0],)), 
                          np.zeros((movie_reviews['train']['negative'].shape[0],))])
test_words = pd.concat([movie_reviews['test']['positive']['reviewText'],
                     movie_reviews['test']['negative']['reviewText']])
y_test = np.concatenate([np.ones((movie_reviews['test']['positive'].shape[0],)), 
                          np.zeros((movie_reviews['test']['negative'].shape[0],))])

In [ ]:
# train_words = pd.concat([movie_reviews['train']['positive']['words'],
#                      movie_reviews['train']['negative']['words']])
# y_train = np.concatenate([np.ones((movie_reviews['train']['positive'].shape[0],)), 
#                           np.zeros((movie_reviews['train']['negative'].shape[0],))])
# test_words = pd.concat([movie_reviews['test']['positive']['words'],
#                      movie_reviews['test']['negative']['words']])
# y_test = np.concatenate([np.ones((movie_reviews['test']['positive'].shape[0],)), 
#                           np.zeros((movie_reviews['test']['negative'].shape[0],))])

In [ ]:
SPARSE = True

if SPARSE:
    # Optimization: add the review length while keeping sparse matrix
    tf_train = tfidf.fit_transform(train_words)
    tf_test = tfidf.transform(test_words)
else:
    tf_train = tfidf.fit_transform(train_words).todense()
    tf_test = tfidf.transform(test_words).todense()

In [ ]:
# print(len(tfidf.vocabulary_))
# tfidf.vocabulary_

## Add review length to modeling input

In [ ]:
ADD_LENGTH = False

if ADD_LENGTH:
    if SPARSE:
        # Hack: pick an existing word to store the count
        len_idx = 0
        test_lengths = [len(words) for words in test_words]

        for idx,words in enumerate(train_words):
            tf_train[idx][len_idx] = len(words)
        for idx,words in enumerate(test_words):
            tf_test[idx][len_idx] = len(words)
        X_train = tf_train
        X_test = tf_test
    else:
        train_lengths = np.array([len(words) for words in train_words]).reshape(-1,1)
        test_lengths = np.array([len(words) for words in test_words]).reshape(-1,1)
        X_train = np.concatenate([tf_train, train_lengths],axis=1)
        X_test = np.concatenate([tf_test, test_lengths],axis=1)
else:
    X_train = tf_train
    X_test = tf_test

### Test and save

In [ ]:
if X_train.shape[0] != y_train.shape[0] or X_test.shape[0] != y_test.shape[0]:
    print('@@@ Problem! @@@')
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

In [ ]:
if False:
    pickle_out = open(data_path 
                      + 'tfidf_' 
                      + str(X_train.shape[0]) + 'Pos_Neg_Samples_'
                      + str(X_train.shape[1]) + 'Feats.pkl'
                      ,"wb")
    pickle.dump(tfidf, pickle_out)
    pickle_out.close()

## Gradient Boosting Classifier for Base

In [ ]:
# Gradient Boosting Classifier parameters
# N_TREES = math.floor(np.sqrt(NB_SAMPLES) * 1.2)
N_TREES = 500
LEARN_RATE = 0.2
MAX_DEPTH = 8
MIN_IN_LEAF = 5 #7
MAX_FEATURES = 'sqrt'

In [ ]:
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                max_depth=MAX_DEPTH,
                                max_features=MAX_FEATURES)

In [ ]:
gbc.fit(X_train, y_train)

In [ ]:
if False:
    pickle.dump(gbc, open(data_path + 'GBC_'
                       + str(NB_SAMPLES) + '_samples_'
                       + str(N_TREES) + '_trees_' 
                       + str(LEARN_RATE) + '_lr_' 
                       + str(MAX_DEPTH) + '_maxdpth_'
                       + str(MIN_IN_LEAF) + '_minleaf_'
                       + str(MAX_FEATURES) + '_feats_'
                       + '.pkl', 'wb'))

In [ ]:
%reload_ext autoreload

print(MAX_FEATURES, ' features', N_TREES,'trees; ',
      LEARN_RATE,'learn_rate; ', MAX_DEPTH, 'max_dpth; ',
      MIN_IN_LEAF, 'min_in_leaf')
classifier_report(gbc, X_train, y_train,
                  'Gradient Boosting Classifier on training set')
classifier_report(gbc, X_test, y_test, 
                  'Gradient Boosting Classifier on test set')

In [ ]:
print('SAMPLE_FRACTION:', SAMPLE_FRACTION,'ADD_LENGTH:',ADD_LENGTH,' SPARSE:',SPARSE,' MAX_FEATURES:',MAX_FEATURES)

### Model tuning
Find the maximum size the machine will take (samples * features)
Accuracy increases by 2% when increasing dataset from 36K to 72K samples

### Investigate what went wrong

In [ ]:
test_pred = gbc.predict(X_test)
true_0_pred_1 = (test_pred == 1) & (y_test == 0)
print(np.unique(true_0_pred_1, return_counts=True))
print(len(true_0_pred_1))
print(len(X_test))
X_test[true_0_pred_1][:10]

In [ ]:
test_reviews = pd.concat([movie_reviews['test']['positive'],
                     movie_reviews['test']['negative']])
test_reviews[true_0_pred_1]['reviewText'][:10]

## Grid search

In [ ]:
if False:
    gb_pipe = Pipeline([('vect', tfidf), ('gb', gbc)])
    gb_pipe.fit(X_train, y_train)
    pickle.dump(gb_pipe, open('pickles/GBCpipe_balanced_comments_'
                           + str(N_TREES) + '_trees_' 
                           + str(LEARN_RATE) + '_lr_' 
                           + str(MAX_DEPTH) + '_maxdpth_'
                           + str(MIN_IN_LEAF) + '_minleaf_'
                           + str(MAX_FEATURES) + '_feats_'
                           + '.pkl', 'wb'))
else:
#     pickle_in = open("pickles/GBC_balanced_comments_300_trees_0.1_lr_15_maxdpth_2_minleaf_20000_feats_.pkl",
#                      "rb")
#     gb_pipe = pickle.load(pickle_in)

In [ ]:
if True:
    grid = {
        'learning_rate': [.1,0.2,0.3],
        'max_depth': [8],
        'min_samples_leaf': [5],
        'max_features': ['sqrt'],
        'n_estimators': [300],
        'random_state': [0]
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)

gbc_grid_cv = GridSearchCV(
    GradientBoostingClassifier(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)

In [ ]:
y_pred = gbc_grid_cv.predict(X_test)

In [ ]:
print('SAMPLE_FRACTION:', SAMPLE_FRACTION,'ADD_LENGTH:',ADD_LENGTH,' SPARSE:',SPARSE,' MAX_FEATURES:',MAX_FEATURES)

In [ ]:
print(gbc_grid_cv.best_params_)
print(gbc_grid_cv.best_score_)
res_df = pd.DataFrame(gbc_grid_cv.cv_results_)
res_df

# Case B: with objective sentences removed

## Split comments into separate sentences

In [ ]:
from nltk.tokenize import sent_tokenize
small['sentence'] = small['reviewText'].map(sent_tokenize)

In [ ]:
small.columns

In [ ]:
small.drop(['reviewerName', 'helpful', 'reviewText', 'summary', 
            'unixReviewTime', 'reviewTime'], axis=1, inplace=True)

In [ ]:
small.shape

In [ ]:
sentences = small['sentence'] \
.apply(pd.Series) \
.merge(small, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)
sentences.head(3)

## Sentence-level prep & cleaning

In [ ]:
%reload_ext autoreload
from utils import split_n_lower, not_about_support

### Split into words and lower the case

In [ ]:
sentences['words'] = sentences['sentence'].apply(lambda s: split_n_lower(s))

In [ ]:
print(sentences.shape)
sentences.head(3)

### Keep support-related sentences as they probably have impact on rating

In [ ]:
# on_movies_filter = [not_about_support(word) for word in sentences['words']]
sentences_on_movie = sentences #[on_movies_filter]

print('Removing {} records'.format(sentences.shape[0]- sentences_on_movie.shape[0]))

In [ ]:
sentences_on_movie.shape

### Base case: A reviews with objective and subjective sentences

In [ ]:
# Kernel dies here at 50K samples
all_reviews_groups = sentences_on_movie.groupby(['reviewerID','asin'])


In [ ]:
all_reviews_stars = all_reviews_groups['overall'].mean()
all_reviews_stars[:3]

In [ ]:
all_reviews_comments = all_reviews_groups['words'].sum()
print(sentences_on_movie.iloc[0, 4])
print(all_reviews_comments.shape)
print(all_reviews_comments[0])
len(all_reviews_comments)

### Remove objective sentences for case B using obj-subj model

In [ ]:
# pd.set_option('display.max_colwidth', -1)
# sentences_on_movie['sentence']
sentences_on_movie.shape

In [ ]:
sentences_on_movie.head(2)

#### Vectorize along the word space of the obj-subj training set

In [ ]:
tfidf = pickle.load(open('pickles/Obj-Subj_tfidf.pkl', 'rb'))
len(tfidf.vocabulary_)

In [ ]:
sentences_tfidf = tfidf.transform(sentences_on_movie['sentence']).todense()

In [ ]:
sentences_tfidf.shape

#### Apply the obj-subj model

In [ ]:
N_TREES = 100
LEARN_RATE = 0.1
MIN_IN_LEAF = 10
pickle_in = open('pickles/GBC_'+ str(N_TREES) +'_' + str(LEARN_RATE) 
                        +'_' + str(MIN_IN_LEAF) + '_20min.pkl', 'rb')
gb_model = pickle.load(pickle_in)

In [ ]:
y_test = gb_model.predict(sentences_tfidf)
len(y_test)

In [ ]:
subjective_sentences = sentences_on_movie[y_test == 1]

In [ ]:
display(Markdown('### Removing {} objective sentences'
                 .format(len(y_test) - len(subjective_sentences))))

In [ ]:
subjective_sentences.head(2)

#### Merge the sentences back into paragraph reviews

In [ ]:
subj_groups = subjective_sentences.groupby(['reviewerID','asin'])
subj_reviews_stars = subj_groups['overall'].mean()
# subjective_reviewssubjective_reviews['sentence'].apply(lambda x: x.sum())
# subjective_reviews_reviews = 
subj_reviews_stars[:3]

In [ ]:
subj_review_comments = subj_groups['words'].sum()
print(subj_review_comments.shape)
print(subj_review_comments[0])
subj_review_comments[:3]

### Check that stars still correspond to the right movie

In [ ]:
start = 6000
end = 6010
all_reviews_comments.loc[('A33Z7JTV7SSW9Y', '0718000315')]


In [ ]:
print(all_reviews_stars.loc[('A33Z7JTV7SSW9Y', '0718000315')])
print(sentences_on_movie.loc[sentences_on_movie['reviewerID']=='A33Z7JTV7SSW9Y']) 
# and sentences_on_movie['asin']=='0718000315'])

In [ ]:
pd.options.display.max_colwidth = -1
print(small.loc[small['reviewerID']=='A33Z7JTV7SSW9Y'])

In [ ]:
sentences_on_movie[start:end]

## Create emotion vectors

In [ ]:
print('Total number of reviews:', all_reviews_comments.shape[0])
print('Total number of subjective reviews:', subj_review_comments.shape[0])

In [ ]:
%reload_ext autoreload
from emotions_seven import Emotions7
emote = Emotions7()

In [ ]:
all_reviews_emotions = emote.vectorize(all_reviews_comments)
print(all_reviews_emotions.shape)

In [ ]:
# emote.emotions_in_text

In [ ]:
all_reviews_emotions.shape

In [ ]:
# all_revs_with_emotions = all_reviews_emotions[emote.emotions_in_text == True]

In [ ]:
# print(all_revs_with_emotions.shape)
# all_revs_stars = all_reviews_stars[emote.emotions_in_text]
all_reviews_emotions[0]

In [ ]:
subj_reviews_emotions = emote.vectorize(subj_review_comments)
print(subj_reviews_emotions.shape)
subj_reviews_emotions[0]

## Fit a model on base case (all comments) for star rating prediction

In [ ]:
X_subj_train, X_subj_cv, y_subj_train, y_subj_cv = train_test_split(
    subj_reviews_emotions, subj_reviews_stars, test_size=0.2, random_state=0)
X_subj_train.shape

In [ ]:
gbc_subj = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                random_state=0)
gbc_subj.fit(X_subj_train, y_subj_train)

In [ ]:
print('Gradient Boosting Classifier')
print('Training score using all comments: {0:.2f}'
      .format(gbc_all.score(X_train, y_train)))
print('CV score using all comments: {0:.2f}'
      .format(gbc_all.score(X_cv, y_cv)))
print('')

# print('Training score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_train, y_subj_train)))
# print('CV score using subjective comments only: {0:.2f}'
#       .format(gbc_subj.score(X_subj_cv, y_subj_cv)))


## Other techniques

In [ ]:
# from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(random_state=0, solver='lbfgs',
#                        multi_class='multinomial',max_iter=1000)
# lr.fit(X_subj_train, y_subj_train)
# print(lr.score(X_subj_train, y_subj_train))
# print(lr.score(X_subj_cv, y_subj_cv))

In [ ]:
ols_all = sm.OLS(y_train, X_train)
results_all = ols_all.fit()
results_all.summary()

In [ ]:
ols_subj = sm.OLS(y_subj_train, X_subj_train)
results_subj = ols_subj.fit()
results_subj.summary()

In [ ]:
import seaborn as sns

all_reviews_emotions, all_reviews_stars

sns.heatmap(raw_df.corr(), annot=True)